# DATA COLLECTION

Installazione librerie necessarie
odfpy: gestione in python degli opendocument

In [1]:
pip install odfpy

You should consider upgrading via the 'C:\Users\Giovanni\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.



     -------------------------------------- 717.0/717.0 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for odfpy, since package 'wheel' is not installed.
  Running setup.py install for odfpy: started
  Running setup.py install for odfpy: finished with status 'done'


Importazione librerie

In [ ]:
import pandas as pd
from io import BytesIO
import numpy as np
import os
from google.colab import drive

Recupero i dataset dal progetto github

In [2]:
!git clone https://github.com/morkioj/parco_veicoli.git

^C


Cloning into 'parco_veicoli'...


**Importazione files ods**
* Per ogni file presente nella cartella estraggo il contenuto del foglio "AV per regione" e da questo genero un dataframe pandas
* A questo df, aggiungo la colonna 'Anno' estrando il valore dal nome del file
* Inserisco il df in una lista di df
* Iterati tutti i file nella cartella, unisco tutti i df presenti nella lista in un nuovo dataframe (df_orig)

In [ ]:
#drive.mount('/content/drive')
#folder_path="/content/drive/MyDrive/Colab Notebooks/dataset/circolante"
folder_path="/content/parco_veicoli/dataset/parco_veicoli_circolante/"
file_path=""
sheet_name="AV_per_regione"
sheet_name2="AV per regione"
df_list=[]

file_list = os.listdir(folder_path)

for file_name in file_list:
  file_path = os.path.join(folder_path,file_name)
  #Ottengo lo stream binario (rb: read binary) del contenuto del file per passarlo come parametro a pd.read
  with open(file_path, 'rb') as f:
    file_content = f.read()
  file_stream=BytesIO(file_content)
  # Per compatibilità con la recente versione di pandas, anzichè passargli il percorso, gli passo il file_stream
    # engine='odf' -> foglio di calcolo openoffice
    # header=1 -> salta la prima riga ed utilizza la seconda come intestazione colonne
  # Utilizzo il try-except perche alcuni file hanno il foglio con _ nel nome mentre altri hanno lo spazio
  try:
    df_file =pd.read_excel(io=file_stream,sheet_name=sheet_name,engine='odf',header=1)
  except ValueError:
    df_file =pd.read_excel(io=file_stream,sheet_name=sheet_name2,engine='odf',header=1)

  #estraggo l'anno dal nome del file e lo inserisco in una nuova colonna
  df_file['ANNO'] = int(file_name.split('_')[-1].replace('.ods',''))

  #Inserisco in dataframe nella lista
  df_list.append(df_file)
  print(f"{file_name} importato...")

#Una volta ottenuti tutti i dataframe (uno per ogni file) ed inseriti nella lista, li concateno tutti in una nuova lista
df_orig = pd.concat(df_list, ignore_index=True)


# DATA CLEANING

**PRIMA PARTE**

1.   Sostituisco i campi nulli delle colonne 'Regione' e 'Alimentazione' con il valore contenuto nel record precedente
2.   Sostituisco i campi nulli delle colonne numeriche con 0
3.   Converto i float in int

In [ ]:

df_opt=df_orig.copy()
#1
df_opt['REGIONE']=df_opt['REGIONE'].ffill()
df_opt['ALIMENTAZIONE']=df_opt['ALIMENTAZIONE'].ffill()
#2
columns_to_fill = ['EURO 0', 'EURO 1', 'EURO 2', 'EURO 3', 'EURO 4', 'EURO 5', 'EURO 6', 'Non contemplato', 'Non identificato']
df_opt[columns_to_fill] = df_opt[columns_to_fill].replace({pd.NA: 0, np.nan: 0})

#3
# df.select_dtypes(include=['float'] ottengo tutte le colonne di tipo float
# .colums e ne prendo i nomi
# {col: 'int' for col in ...} dictionary comprehension: crea un dizionario dove le chiavi sono i nomi delle colonne ed i valori 'int'
# quindi per ogni colonna float del df crea una coppia del dizionario nella forma <nomecolonna;'int'>
df_opt = df_opt.astype({col: 'int' for col in df_opt.select_dtypes(include=['float']).columns})

4. Rimuovo i subtotali regionali (Campo Regione contiene 'Totale')
5. Rimuovo la suddivisione per fascia di cilindrata (Campo Alimentazione non contiene 'Totale')
6. Rimuovo la colonna 'FASCIA' in conseguenza del punto 4
7. Rimuovo la parola 'Totale' dalla descrizione delle alimentazioni

In [ ]:
# Ricerca di testo all'interno del campo (case insensitive e nan da considerarsi false)
# 4
df_filt = df_opt[~df_opt['REGIONE'].str.contains('totale', case=False, na=False)]
#5
df_filt = df_filt[df_filt['ALIMENTAZIONE'].str.contains('totale', case=False, na=False)].reset_index(drop=True)
#6
df_filt=df_filt.drop(columns=['FASCIA'],axis=1)
#7
df_filt['ALIMENTAZIONE']=df_filt['ALIMENTAZIONE'].str.replace(" totale","",case=False)


**SECONDA PARTE**

1. Assimilo i veicoli senza classe di inquinamento nella EURO 0 ([NON IDENTIFICATO]==0 --> EURO 0)
2. Sposto i veicoli con alimentazione 'non definita' oppure 'altre' in una nuova categoria 'altro' ([Alimentazione]== 'Non Definito' || 'Altre' --> 'Altro')
3. Rimuovo i record per i quali non è definita la regione ([REGIONE]== 'Non Definito' --> RIMUOVERE)   

Stampa dei valori che saranno rimossi/aggregati/modificati

In [ ]:
#'REGIONE' == NON DEFINITO
print (df_filt[df_filt['REGIONE']=='NON DEFINITO'].to_string())

# Record con 'NON IDENTIFICATO' != 0
print (df_filt[df_filt['Non identificato']!=0].to_string())

#'ALIMENTAZIONE' == NON DEFINITO
print (df_filt[df_filt['ALIMENTAZIONE']=='NON DEFINITO'].to_string())

In [ ]:
#1
# Creo un df temporaneo in cui inserire i record da modificare
df_tmp = df_filt[df_filt['Non identificato'] != 0]
# Sommo i valori: Seleziono le sole righe relative agli indici di dt_tmp e la colonna 'EURO 0' su cui sommare
df_filt.loc[df_tmp.index, 'EURO 0'] += df_tmp['Non identificato']
#Rimuovo la colonna "non identificato"
df_filt = df_filt.drop(columns=['Non identificato'])

#print (df_filt.to_string())

In [ ]:
#2
#CREO UN DATASET TEMPORANEO IN CUI INSERISCO LE NUOVE RIGHE (con ALIMENTAZIONE=ALTRO) il cui valore è dato dalla somma delle righe con alimentazione=ALTRE || NON DEFINITO
df_tmp = df_filt[df_filt['ALIMENTAZIONE'].isin(['ALTRE','NON DEFINITO'])].groupby(['REGIONE','ANNO']).sum(numeric_only=True).reset_index()
df_tmp.insert(loc=1,column='ALIMENTAZIONE',value='ALTRO')
#RIMUOVO LE RIGHE DA DF DI PARTENZA DOVE ALIMENTAZIONE = ALTRE o = NON DEFINITO
df_filt.drop(df_filt[df_filt['ALIMENTAZIONE'].isin(['ALTRE', 'NON DEFINITO'])].index, inplace=True)

#3
#RIMUOVO LE RIGHE DA DF DI PARTENZA DOVE REGIONE = 'NON DEFINITO"
df_filt.drop(df_filt[df_filt['REGIONE'] == 'NON DEFINITO'].index, inplace=True)

#UNISCO IL TUTTO IN UN NUOVO DATAFRAME (Altrimenti una volta alterato questo blocco di codice non troverebbe piu oggetti da lavorare)
df_wrk = pd.concat([df_filt,df_tmp],ignore_index=True)


# PREPROCESSING

Partendo dalla tipologia di alimentazione, aggiungo il campo 'TIPOLOGIA-MOTORE' per rappresentare la classe di motorizzazione (ENDOTERMICO - ELETTRICO - IBRIBO)

In [ ]:
# Valori distinti dei tipi di alimentazione
print(df_wrk['ALIMENTAZIONE'].unique())

In [ ]:
# Creo un dizionario per mappare la classe di motorizzazione del veicolo
engine_map ={
  'BENZINA': 'ENDOTERMICO',
	'BENZINA E GAS LIQUIDO': 'ENDOTERMICO',
	'BENZINA E METANO': 'ENDOTERMICO',
	'ELETTRICITA': 'ELETTRICO',
	'GASOLIO': 'ENDOTERMICO',
	'GASOLIO E GAS': 'ENDOTERMICO',
	'IBRIDO BENZINA': 'IBRIDO',
	'IBRIDO GASOLIO': 'IBRIDO',
	'METANO': 'ENDOTERMICO',
	'ELETTRICO-IBRIDO': 'ELETTRICO',
	'ALTRO': 'ENDOTERMICO'
}

#aggiungo la colonna al dataframe per la rappresentazione della variabile tipologia motore, in posizione 2 mettendo in join la colonna "alimentazione" con il dizionario sopra
df_wrk.insert(2,'TIPOLOGIA_MOTORE',df_wrk['ALIMENTAZIONE'].map(engine_map).fillna('NON TROVATA'))

Partendo dalla regione, aggiungo il campo 'AREA_GEOGRAFICA' (NORD-EST, NORD-OVEST,CENTRO,SUD,ISOLE)

In [ ]:
zone_map={
  'ABRUZZO': 'SUD',
  'BASILICATA': 'SUD',
  'CALABRIA': 'SUD',
  'CAMPANIA': 'SUD',
  'EMILIA ROMAGNA': 'NORD-EST',
  'FRIULI VENEZIA GIULIA': 'NORD-EST',
  'LAZIO': 'CENTRO',
  'LIGURIA': 'NORD-OVEST',
  'LOMBARDIA': 'NORD-OVEST',
  'MARCHE': 'CENTRO',
  'MOLISE': 'SUD',
  'PIEMONTE': 'NORD-OVEST',
  'PUGLIA': 'SUD',
  'SARDEGNA': 'ISOLE',
  'SICILIA': 'ISOLE',
  'TOSCANA': 'CENTRO',
  'TRENTINO ALTO ADIGE': 'NORD-EST',
  'UMBRIA': 'CENTRO',
  'VALLE D\'AOSTA': 'NORD-OVEST',
  'VENETO': 'NORD-EST'
}
df_wrk.insert(0,'AREA_GEOGRAFICA',df_wrk['REGIONE'].map(zone_map).fillna('NON TROVATA'))

Ottimizzo il dataframe finale secondo le esigenze della rappresentazione grafica

In [ ]:
#Raggruppo per regione,tipologia ed anno sommando il totale veicoli (rimuovendo di conseguenza tutti i campi superflui)
df_graph = df_wrk.groupby(['AREA_GEOGRAFICA','REGIONE','TIPOLOGIA_MOTORE','ANNO'])['TOTALE'].sum().reset_index()
#Rimuovo la "regione" italia
df_graph = df_graph[df_graph['REGIONE'] != 'ITALIA']

#esporto in excel
df_graph.to_excel("/content/circolante_powerbi.xlsx")